In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [2]:
import data.ImageData as ImageData
import models.SUNSET as SUNSET
import functions.utils as utils

In [3]:
# define folder location

dir_path = os.getcwd()
data_folder = os.path.join(dir_path, "data")

#define file location

images_trainval_path = os.path.join(data_folder,'images_trainval.npy')
pv_log_trainval_path = os.path.join(data_folder,'pv_log_trainval.npy')
datetime_trainval_path = os.path.join(data_folder,'datetime_trainval.npy')

images_test_path = os.path.join(data_folder,'images_test.npy')
pv_log_test_path = os.path.join(data_folder,'pv_log_test.npy')
datetime_test_path = os.path.join(data_folder,'datetime_test.npy')

In [4]:
# load PV output and images for the trainval set

pv_log_trainval = np.load(pv_log_trainval_path)
images_trainval = np.load(images_trainval_path)
images_trainval = images_trainval.reshape(images_trainval.shape[0],3,64,64)
datetimes_trainval = np.load(datetime_trainval_path, allow_pickle=True)

# load PV output and images for the test set

pv_log_test = np.load(pv_log_test_path)
images_test = np.load(images_test_path)
images_test = images_test.reshape(images_test.shape[0],3,64,64)
datetimes_test = np.load(datetime_test_path, allow_pickle=True)


#  ATENTION !!!! Create a dev dataset version ***************************************************************
pv_log_trainval = pv_log_trainval[0:2000]
images_trainval = images_trainval[0:2000]
datetimes_trainval = datetimes_trainval[0:2000]

pv_log_test = pv_log_test[0:400]
images_test = images_test[0:400]
datetimes_test = datetimes_test[0:400]
#  ATENTION !!!! ********************************************************************************************


In [5]:
# Create the Dataset classes

train_data = ImageData.ImageData(torch.tensor(images_trainval), torch.tensor(pv_log_trainval))
test_data = ImageData.ImageData(torch.tensor(images_test),torch.tensor(pv_log_test))

# Dataloaders

batch_size = 256
batch_size_eval = 256

indices = list(range(train_data.__len__()))
n_valid = train_data.__len__() // 5 # 20% train data

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(indices[:-n_valid]), # we need to split data before that
    #num_workers=1,
    #pin_memory=use_cuda
)

valid_loader = DataLoader(
    train_data,
    batch_size=batch_size_eval,
    sampler=SubsetRandomSampler(indices[-n_valid:]), # we need to split data before that
    #num_workers=1,
    #pin_memory=use_cuda,
)

test_loader = DataLoader(
    test_data,
    batch_size=batch_size_eval,
    #num_workers=1,
    #pin_memory=use_cuda,
)

In [6]:
### We initialize the parameters

num_epochs = 5
learning_rate = 3e-6

loss_fn = nn.MSELoss()

#model = SUNSET.SUNSETBase()
model = SUNSET.SUNSET_Sunny()
#model = SUNSET.SUNSET_Cloudy()
#model = SUNSET.SUNSET_Overcast()


optimizer = optim.Adam(model.parameters() , lr=learning_rate)

Losses = []
Tlosses = []

for epoch in range(num_epochs):
    
    Losses.append(utils.train(model, train_loader, optimizer, loss_fn).item())    
    Tlosses.append(utils.test(model, train_loader, loss_fn).item())
    print(epoch)

C:\Users\Marcos\Documents\UdeM\IFT-6759-Photovoltaic-forecast\data\ImageData.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(self.inputs[index])
C:\Users\Marcos\Documents\UdeM\IFT-6759-Photovoltaic-forecast\data\ImageData.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.labels[index])
C:\Users\Marcos\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Ma

0
1
2
3
4


In [ ]:
Losses

In [ ]:
Tlosses

In [ ]:
#Losses = list(map(int, Losses))
#Tlosses = list(map(int, Tlosses))
#plt.plot(range(num_epochs), Losses)
#plt.plot(range(num_epochs), Tlosses)
#plt.xlabel('epochs')
#plt.ylabel('mean squared loss')